 # Raw SQL Playground

 This notebook demonstrates direct database access using Python's DB-API
 (`sqlite3`) without any ORM or SQL abstraction layer.

 Goals:
 - work with connections and cursors
 - execute raw SQL statements
 - understand transactions and commits

 ## Imports

 We use the low-level sqlite3 module and a small helper
 to open database connections.

In [ ]:
import sqlite3

from db import get_connection

<details>
<summary><strong style="font-size:1.5em;">Raw SQL – Connection, Cursor, Execute, Commit (Mental Model)</strong></summary>

<br>

### 1. Connection (`connect`)
- Opens a connection to the database file  
- Reserves resources (and a file lock in SQLite, especially on Windows)  
- Manages transactions (begin / commit / rollback)  
- A connection alone does **not** execute SQL  

---

### 2. Cursor (`cursor`)
- Creates an execution context for SQL statements  
- Sends SQL commands to the database and reads results  
- Multiple cursors can exist on the same connection  
- Think of it as a **SQL editor + result reader**

---

### 3. Execute (`execute`)
- Sends a SQL command to the database  
- The database parses and runs the statement  
- For `SELECT`: results are prepared, not yet transferred to Python  
- For `INSERT / UPDATE / DELETE`: changes are staged in a transaction  

---

### 4. Fetching results (`fetchone`, `fetchmany`, `fetchall`)
- Explicitly transfers result rows from the database to Python  
- Required after `SELECT`  
- Not used for write operations  

---

### 5. Commit (`commit`)
- Finalizes the current transaction  
- Makes all changes permanent  
- Without `commit()`, changes are rolled back on close  
- Required after any write operation  

---

### 6. Close (`close`)
- Closes the database connection  
- Releases file locks and resources  
- Ends any open transaction (rollback if not committed)

---

### Cursor vs Connection closing
- `cursor.close()` closes only the cursor  
- `conn.close()` closes the entire connection and all cursors  
- In practice: **always close the connection**

---

### If you forget to close
- Database file may stay locked  
- Resources remain allocated  
- Uncommitted changes are lost

</details>


 ## Open connection and cursor

 - Connection: manages the database session and transactions
 - Cursor: executes SQL statements and fetches results

In [ ]:
conn = get_connection()
cursor = conn.cursor()

 ## Insert sample data

 We insert a category first, then products referencing it.
 Parameter placeholders (`?`) prevent SQL injection.

In [ ]:
cursor.execute(
    "INSERT INTO categories (name) VALUES (?)",
    ("electronics",),
)

category_id = cursor.lastrowid  # ID of the inserted category

cursor.execute(
    "INSERT INTO products (name, price, category_id) VALUES (?, ?, ?)",
    ("Laptop", 1200.0, category_id),
)

cursor.execute(
    "INSERT INTO products (name, price, category_id) VALUES (?, ?, ?)",
    ("Headphones", 199.0, category_id),
)

# Commit makes all inserts permanent
conn.commit()

 ## Simple SELECT

 Fetch all products from the database.

In [ ]:
cursor.execute(
    "SELECT id, name, price FROM products"
)

cursor.fetchall() # .execute happens in db, .fetchall() brings data to python

 ## JOIN products with categories

 Raw SQL JOIN showing the relationship explicitly.

In [ ]:
cursor.execute("""
SELECT
    p.name AS product,
    c.name AS category,
    p.price
FROM products p
JOIN categories c ON p.category_id = c.id
""")

cursor.fetchall()

 ## Aggregation

 Count products per category using GROUP BY.

In [ ]:
cursor.execute("""
SELECT
    c.name,
    COUNT(p.id) AS product_count
FROM categories c
JOIN products p ON p.category_id = c.id
GROUP BY c.name
""")

cursor.fetchall()

 ## Cleanup

 Closing the connection:
 - releases the database file
 - ends the current transaction if still open

In [ ]:
conn.close()